# QuPDE usage examples

QuPDE is a Python library that finds an optimal and monomial quadratic transformation (quadratization) for nonquadratic PDEs using Sympy. QuPDE handles one-dimensional PDEs that are polynomial or rational. We present some usage examples where we showcast some functionalities of our algorithm.

First we import Sympy and QuPDE 

In [ ]:
import sympy as sp
from qupde import *

## General usage
### Dym equation 

The Harry Dym equation describes the dynamics of several physical systems. This equation
is completely integrable and represents a system in which dispersion and nonlinearity are coupled
together: 

$$u_{t}=u^{3}u_{xxx}.$$

Let us find a quadratization for the Dym equation using QuPDE. First, we write the differential equation using Sympy objects:

In [ ]:
t, x = sp.symbols('t x')
u = sp.Function('u')(t,x)

u_t = u**3 * sp.Derivative(u, x, 3)

To find a quadratization we need to provide the algorithm with two parameters: the PDE, and the number of differentiations with respect to the spatial variable to be performed on the auxiliary variables. We set the second parameter as three and we call the main function of the software *quadratize*. This function's first paramater is a list of tuples representing each undefined function with its corresponding differential equation within the PDE system. The second parameter is an integer *n* representing the number of differentiations the algorithm will compute for the new variables. In our example: 

In [ ]:
quadratize([(u, u_t)], 3)

This function returns a tuple with:
- A list with the auxiliary polynomial variables
- A list with the introduced rational variables 
- The number of quadratizations attempted (nodes visited)

Now, if we want to see the quadratization and the transformed PDE in a more readable format, we call the same function but with the optional *printing* parameter with the available printing options: 
- `'pprint'` for pretty printing (Sympy's functionality).
- `'latex'` for printing the result in LaTeX format. 

In [ ]:
quadratize([(u, u_t)], 3, printing='pprint')

In [ ]:
quadratize([(u, u_t)], 3, printing='latex')

## *Quadratize* parameters

The *quadratize* function has other parameters that the user can modify. These correspond to: 
1. Changing the heuristic to sort each set of new variables introduced when searching for an optimal quadratization.
2. Changing the bound of the maximum number of variables to explore (default is 10). 
3. Changing the symbol of the first independent variable (default is *t*). 
4. Changing the maximum order of derivatives allowed within the PDE's quadratic transformation (default is the maximum order in the original PDE). 
5. Changing the search algorithm.

We will offer some examples to show how some of them work. 

### 1. Change sorting heuristic
In the algorithm, there are three heuristics implemented: 
- By order of derivatives and total degree of the monomials (`by_order_degree()`)
- By total degree and order of derivatives of the monomials (`by_degree_order()`). 
- By the function: $degree + 2 \cdot order$ (`by_fun()`).

The default option implemented is `by_fun()`. If we want to use the sorting function `by_order_degree()` to find a quadratization, we run

In [ ]:
quadratize([(u, u_t)], 3, sort_fun=by_order_degree, printing='pprint')

### 2. Change the bound of maximum number of new variables

The default number for this parameter is 10. If we want to quadratize a PDE that is simple in terms of polynomial degrees, we may want to decrease the bound on the number of variables to find the optimal quadratization faster. Or if a PDE has higher degrees, we may want to increase this bound. To do this, we just arbitrarly set the parameter `nvars_bound`. In our example, if we change this bound to 3, we obtain an optimal quadratization faster:

In [ ]:
quadratize([(u, u_t)], 3, nvars_bound=3, printing='pprint')

### 3. Change the symbol for the first independent variable

If we want to change the symbol of the first independent variable, we set the parameter `first_indep` equal to the new symbol. Note that changing the symbol for the second independent variable can be done just by defining the equation in Sympy with this new symbol. 

In [ ]:
z=sp.symbols('z')
y=sp.symbols('y')
v=sp.Function('v')(z,y)

vz = v**3 * sp.Derivative(v, y, 3)

quadratize([(v, vz)], 3, first_indep=z, printing='pprint')

### 4. Change the maximum order of derivatives

To change this parameter, we have to set `max_der_order` to the desired maximum order. An important note is that in some cases, we need to relax this limit to obtain a quadratization for a PDE. For example, if we try running *quadratize* for the equation

In [ ]:
ut2 = u - sp.Derivative(u, (x, 2))**2*u - 2 * sp.Derivative(u, (x, 2))**2 * u - u + v * u**2 - u**3 
quadratize([(u, ut2)], n_diff=4, printing = 'pprint')

the algorithm does not find a quadratization. Now, if we relax the maximum order of derivatives rule and allow derivatives up to order 4, we obtain a different result 

In [ ]:
quadratize([(u, ut2)], n_diff=4, max_der_order=4, printing = 'pprint')

### 5. Change the search algorithm 

To change the search framework to be used, we need to set the parameter `search_alg` to either `'bnb'` for branch-and-bound, or `'inn'` for the incremental nearest neighbor implementation. 

In [ ]:
quadratize([(v, vz)], 3, search_alg='inn', printing='pprint')

## Other examples
### Allen-Cahn equation

First, we run QuPDE for the Allen-Cahn equation, described by the PDE $$u_t = u_{xx} + u - u^3.$$ This time, we print the result in LaTeX format. 

In [ ]:
t, x = sp.symbols('t x')
u = sp.Function('u')(t,x)

u_t = D(u, x, 2) + u - u**3 

quadratize([(u, u_t)], 3, printing='latex')

### FitzHugh-Nagamo system

The FitzHugh-Nagamo system is a simplified neuron model of the Hodgkin-Huxley model, which describes activation and deactivation dynamics of a spiking neuron. Its governing equations are

$$v_t = \epsilon v_{xx} + \dfrac{1}{\epsilon}v(v - 0.1)(1 - v) - \dfrac{1}{\epsilon}u + \dfrac{1}{\epsilon}q,$$
$$u_t = hv - \gamma u + q.$$ 

We define the symbolic coefficients first to use QuPDE on this equation. 

In [ ]:
t, x = sp.symbols('t x')
v = sp.Function('v')(t,x)
y = sp.Function('y')(t,x)
epsilon, h, gamma, r = sp.symbols('epsilon h gamma r', constant=True)

v_t = epsilon * D(v, x, 2) - (1/epsilon) * (v * (v - 0.1) * (1 - v)) - y/epsilon + r/epsilon
y_t = h * v - gamma * y + r

quadratize([(v, v_t), (y, y_t)], 3, search_alg='bnb', printing='pprint')